## GPT-4o Vision Modalityを使用したRetrieval-Augmented Generationの最適化

画像、グラフィック、表が豊富に含まれる文書を扱う際、Retrieval-Augmented Generation（RAG）の実装には独特の課題があります。従来のRAGモデルはテキストデータでは優れた性能を発揮しますが、視覚的要素が情報伝達において重要な役割を果たす場合には、しばしば不十分な結果となります。このクックブックでは、vision modalityを活用して視覚的コンテンツを抽出・解釈することで、そのギャップを埋め、生成される応答が可能な限り情報豊富で正確になるようにします。

私たちのアプローチでは、文書を画像に解析し、メタデータタグを使用して画像、グラフィック、表を含むページを識別します。セマンティック検索でそのようなページが取得された場合、テキストのみに依存するのではなく、ページ画像をvision modelに渡します。この方法により、視覚的データに関するユーザークエリを理解し、回答するモデルの能力が向上します。

このクックブックでは、以下の主要概念を探求し、実演します：

##### 1. Pineconeを使用したVector Storeの設定：
- ベクター埋め込みを効率的に保存するためのPineconeの初期化と設定方法を学習します。

##### 2. PDFの解析と視覚情報の抽出：
- PDFページを画像に変換する技術を発見します。
- GPT-4o vision modalityを使用して、画像、グラフィック、表を含むページからテキスト情報を抽出します。

##### 3. 埋め込みの生成：
- 埋め込みモデルを活用してテキストデータのベクター表現を作成します。
- 視覚的コンテンツを持つページにフラグを立て、vector storeにメタデータフラグを設定し、GPT-4o vision modalityに渡すための画像を取得します。

##### 4. Pineconeへの埋め込みのアップロード：
- これらの埋め込みを保存と検索のためにPineconeにアップロードします。

##### 5. 関連ページのセマンティック検索の実行：
- ページテキストでセマンティック検索を実装し、ユーザーのクエリに最も適合するページを見つけます。
- 一致するページテキストをGPT-4oにコンテキストとして提供し、ユーザーのクエリに回答します。

##### 6. 視覚的コンテンツを含むページの処理（オプションステップ）：
- 追加のコンテキストを使用した質問応答のために、GPT-4o vision modalityを使用して画像を渡す方法を学習します。
- このプロセスが視覚的データに関する応答の精度をどのように向上させるかを理解します。

このクックブックの終わりまでに、複雑な視覚的要素を含む文書を処理・解釈できるRAGシステムの実装について、堅実な理解を得ることができます。この知識により、より豊富で正確な情報を提供するAIソリューションを構築し、ユーザーの満足度とエンゲージメントを向上させることができるようになります。

概念を説明するために、World Bank報告書 - [A Better Bank for a Better World: Annual Report 2024](https://documents1.worldbank.org/curated/en/099101824180532047/pdf/BOSIB13bdde89d07f1b3711dd8e86adb477.pdf) を使用します。この文書には画像、表、グラフィックデータが混在しているためです。

Vision Modalityの使用はリソース集約的であり、レイテンシとコストの増加につながることを念頭に置いてください。プレーンテキスト抽出方法では評価ベンチマークでの性能が不十分な場合にのみ、Vision Modalityを使用することをお勧めします。この背景を踏まえて、詳しく見ていきましょう。

### ステップ1: Pineconeを使用したベクターストアの設定
このセクションでは、Pineconeを使用してベクターストアを設定し、埋め込みを効率的に保存・管理します。Pineconeは高次元ベクターデータの処理に最適化されたベクターデータベースで、セマンティック検索や類似性マッチングなどのタスクに不可欠です。

**前提条件**
1. Pineconeにサインアップし、こちらの手順に従ってAPIキーを取得してください [Pinecone Database Quickstart](https://docs.pinecone.io/guides/get-started/quickstart)
2. `pip install "pinecone[grpc]"`を使用してPinecone SDKをインストールしてください。gRPC（gRPC Remote Procedure Call）は、HTTP/2をトランスポートに使用し、Protocol Buffers（protobuf）をインターフェース定義言語として使用し、分散システムでクライアント・サーバー間通信を可能にする高性能でオープンソースの汎用RPCフレームワークです。マイクロサービスアーキテクチャに適したサービス間通信をより効率的にするように設計されています。

**APIキーの安全な保存**
1. セキュリティ上の理由から、プロジェクトディレクトリの.envファイルにAPIキーを以下のように保存してください：
 `PINECONE_API_KEY=your-api-key-here`
 2. .envファイルからAPIキーを読み取るために`pip install python-dotenv`をインストールしてください。

**Pineconeインデックスの作成**
Pinecone上で埋め込みデータベースを初期化するために`create_index`関数を使用します。考慮すべき重要なパラメータが2つあります：

1. Dimension（次元数）：これは選択したモデルが生成する埋め込みの次元数と一致する必要があります。例えば、OpenAIのtext-embedding-ada-002モデルは1536次元の埋め込みを生成し、text-embedding-3-largeは3072次元の埋め込みを生成します。このクックブックではtext-embedding-3-largeモデルを使用するため、次元数を3072に設定します。

2. Metric（メトリック）：距離メトリックは、ベクター間の類似性がどのように計算されるかを決定します。Pineconeはcosine、dotproduct、euclideanなど複数のメトリックをサポートしています。このクックブックでは、コサイン類似度メトリックを使用します。距離メトリックについて詳しくは、[Pinecone Distance Metrics documentation](https://docs.pinecone.io/guides/indexes/understanding-indexes#distance-metrics)をご覧ください。

In [9]:
import os
import time
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key)

# Create a serverless index
index_name = "my-test-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

[Pinecone](https://app.pinecone.io/)のIndexesリストに移動すると、インデックスのリストに`my-test-index`が表示されるはずです。

### ステップ2: PDFの解析と視覚的情報の抽出:

このセクションでは、世界銀行のレポート - [A Better Bank for a Better World: Annual Report 2024](https://documents1.worldbank.org/curated/en/099101824180532047/pdf/BOSIB13bdde89d07f1b3711dd8e86adb477.pdf) のPDF文書を解析し、画像、グラフィック、表の説明など、テキストと視覚的情報を抽出します。このプロセスには3つの主要なステップが含まれます：

1. **PDFを個別のページに解析する:** 処理を簡単にするため、PDFを個別のページに分割します。
2. **PDFページを画像に変換する:** これにより、GPT-4oのビジョン機能がページを画像として分析できるようになります。
3. **画像と表を処理する:** GPT-4oにテキストを抽出し、文書内の画像、グラフィック、表を説明するよう指示を提供します。

**前提条件**

続行する前に、以下のパッケージがインストールされていることを確認してください。また、OpenAI APIキーが環境変数として設定されていることを確認してください。PDF レンダリングのためにPopplerのインストールも必要な場合があります。

`pip install PyPDF2 pdf2image pytesseract pandas tqdm`

**ステップの詳細:**

**1. PDFのダウンロードとチャンク化:**
- `chunk_document`関数は、提供されたURLからPDFをダウンロードし、PyPDF2を使用して個別のページに分割します。
- 各ページは、リスト内の個別のPDFバイトストリームとして保存されます。

**2. PDFページの画像への変換:**
- `convert_page_to_image`関数は、単一ページのPDFバイトを受け取り、pdf2imageを使用して画像に変換します。
- 画像は、さらなる処理のために'images'ディレクトリにローカルに保存されます。

**3. GPT-4oビジョンモダリティを使用したテキスト抽出:**
- `extract_text_from_image`関数は、GPT-4oのビジョン機能を使用してページの画像からテキストを抽出します。
- この方法は、スキャンされた文書からでもテキスト情報を抽出できます。
- このモダリティはリソース集約的であるため、より高いレイテンシとコストが関連することに注意してください。

**4. 文書全体の処理:**
- `process_document`関数は、各ページの処理を統括します。
- 処理状況を表示するためにプログレスバー（tqdm）を使用します。
- 各ページから抽出された情報はリストに収集され、その後Pandas DataFrameに変換されます。

In [7]:
import base64
import requests
import os
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_bytes
from io import BytesIO
from openai import OpenAI
from tqdm import tqdm

# Link to the document we will use as the example 
document_to_parse = "https://documents1.worldbank.org/curated/en/099101824180532047/pdf/BOSIB13bdde89d07f1b3711dd8e86adb477.pdf"

# OpenAI client 
oai_client = OpenAI()


# Chunk the PDF document into single page chunks 
def chunk_document(document_url):
    # Download the PDF document
    response = requests.get(document_url)
    pdf_data = response.content

    # Read the PDF data using PyPDF2
    pdf_reader = PdfReader(BytesIO(pdf_data))
    page_chunks = []

    for page_number, page in enumerate(pdf_reader.pages, start=1):
        pdf_writer = PdfWriter()
        pdf_writer.add_page(page)
        pdf_bytes_io = BytesIO()
        pdf_writer.write(pdf_bytes_io)
        pdf_bytes_io.seek(0)
        pdf_bytes = pdf_bytes_io.read()
        page_chunk = {
            'pageNumber': page_number,
            'pdfBytes': pdf_bytes
        }
        page_chunks.append(page_chunk)

    return page_chunks


# Function to encode the image
def encode_image(local_image_path):
    with open(local_image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


# Function to convert page to image     
def convert_page_to_image(pdf_bytes, page_number):
    # Convert the PDF page to an image
    images = convert_from_bytes(pdf_bytes)
    image = images[0]  # There should be only one page

    # Define the directory to save images (relative to your script)
    images_dir = 'images'  # Use relative path here

    # Ensure the directory exists
    os.makedirs(images_dir, exist_ok=True)

    # Save the image to the images directory
    image_file_name = f"page_{page_number}.png"
    image_file_path = os.path.join(images_dir, image_file_name)
    image.save(image_file_path, 'PNG')

    # Return the relative image path
    return image_file_path


# Pass the image to the LLM for interpretation  
def get_vision_response(prompt, image_path):
    # Getting the base64 string
    base64_image = encode_image(image_path)

    response = oai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
    )
    return response


# Process document function that brings it all together 
def process_document(document_url):
    try:
        # Update document status to 'Processing'
        print("Document processing started")

        # Get per-page chunks
        page_chunks = chunk_document(document_url)
        total_pages = len(page_chunks)

        # Prepare a list to collect page data
        page_data_list = []

        # Add progress bar here
        for page_chunk in tqdm(page_chunks, total=total_pages, desc='Processing Pages'):
            page_number = page_chunk['pageNumber']
            pdf_bytes = page_chunk['pdfBytes']

            # Convert page to image
            image_path = convert_page_to_image(pdf_bytes, page_number)

            # Prepare question for vision API
            system_prompt = (
                "The user will provide you an image of a document file. Perform the following actions: "
                "1. Transcribe the text on the page. **TRANSCRIPTION OF THE TEXT:**"
                "2. If there is a chart, describe the image and include the text **DESCRIPTION OF THE IMAGE OR CHART**"
                "3. If there is a table, transcribe the table and include the text **TRANSCRIPTION OF THE TABLE**"
            )

            # Get vision API response
            vision_response = get_vision_response(system_prompt, image_path)

            # Extract text from vision response
            text = vision_response.choices[0].message.content

            # Collect page data
            page_data = {
                'PageNumber': page_number,
                'ImagePath': image_path,
                'PageText': text
            }
            page_data_list.append(page_data)

        # Create DataFrame from page data
        pdf_df = pd.DataFrame(page_data_list)
        print("Document processing completed.")
        print("DataFrame created with page data.")

        # Return the DataFrame
        return pdf_df

    except Exception as err:
        print(f"Error processing document: {err}")
        # Update document status to 'Error'


df = process_document(document_to_parse)

Document processing started


Processing Pages: 100%|██████████| 49/49 [18:54<00:00, 23.14s/it]

Document processing completed.
DataFrame created with page data.


DataFrameを調べて、ページが正しく処理されていることを確認しましょう。簡潔にするため、最初の5行のみを取得して表示します。また、'images'ディレクトリに生成されたページ画像も確認できるはずです。

In [10]:
from IPython.display import display, HTML

# Convert the DataFrame to an HTML table and display top 5 rows 
display(HTML(df.head().to_html()))

サンプルページとして、埋め込まれたグラフィックとテキストを含むページ21を見てみましょう。視覚モダリティが視覚情報を効果的に抽出し、記述していることが観察できます。例えば、このページの円グラフは以下のように正確に記述されています：

`"FIGURE 6: MIDDLE EAST AND NORTH AFRICA IBRD AND IDA LENDING BY SECTOR - FISCAL 2024 SHARE OF TOTAL OF $4.6 BILLION" is a circular chart, resembling a pie chart, illustrating the percentage distribution of funds among different sectors. The sectors include:`

In [11]:
# Filter and print rows where pageNumber is 21
filtered_rows = df[df['PageNumber'] == 21]
for text in filtered_rows.PageText:
    print(text)

**TRANSCRIPTION OF THE TEXT:**

We also committed $35 million in grants to support emergency relief in Gaza. Working with the World Food Programme, the World Health Organization, and the UN Children’s Fund, the grants supported the delivery of emergency food, water, and medical supplies. In the West Bank, we approved a $200 million program for the continuation of education for children, $22 million to support municipal services, and $45 million to strengthen healthcare and hospital services.

**Enabling green and resilient growth**
To help policymakers in the region advance their climate change and development goals, we published Country Climate and Development Reports for the West Bank and Gaza, Lebanon, and Tunisia. In Libya, the catastrophic flooding in September 2023 devastated eastern localities, particularly the city of Derna. The World Bank, together with the UN and the European Union, produced a Rapid Damage and Needs Assessment to inform recovery and reconstruction efforts.

W

### ステップ3: エンベディングの生成

このセクションでは、文書の各ページから抽出されたテキストコンテンツをベクトルエンベディングに変換することに焦点を当てます。これらのエンベディングはテキストの意味的な意味を捉え、効率的な類似性検索や様々な自然言語処理（NLP）タスクを可能にします。また、画像、グラフィック、表などの視覚的要素を含むページを特定し、特別な処理のためにフラグを立てます。

**ステップの詳細:**

**1. 視覚的コンテンツのフラグ追加**   
  
視覚的情報を含むページを処理するため、ステップ2では視覚モダリティを使用してチャート、表、画像からコンテンツを抽出しました。プロンプトに特定の指示を含めることで、視覚的コンテンツを説明する際にモデルが`DESCRIPTION OF THE IMAGE OR CHART`や`TRANSCRIPTION OF THE TABLE`などのマーカーを追加することを確実にします。このステップでは、そのようなマーカーが検出された場合、Visual_Input_Processedフラグを'Y'に設定し、そうでなければ'N'のままにします。

視覚モダリティは大部分の視覚的情報を効果的に捉えますが、特に工学図面のような複雑な視覚要素では、翻訳過程で一部の詳細が失われる可能性があります。ステップ6では、このフラグを使用して、ページの画像を追加のコンテキストとしてGPT-4 Visionに渡すタイミングを決定します。これはRAGソリューションの効果を大幅に向上させることができるオプションの機能強化です。

**2. OpenAIのエンベディングモデルによるエンベディング生成**  

各ページの意味的コンテンツを表現する高次元エンベディングを生成するために、OpenAIのエンベディングモデル`text-embedding-3-large`を使用します。

注意: 使用するエンベディングモデルの次元数が、Pineconeベクトルストアの設定と一致していることを確認することが重要です。今回の場合、`text-embedding-3-large`のデフォルト次元数に合わせて、Pineconeデータベースを3072次元で設定しました。

In [112]:
# Add a column to flag pages with visual content
df['Visual_Input_Processed'] = df['PageText'].apply(
    lambda x: 'Y' if 'DESCRIPTION OF THE IMAGE OR CHART' in x or 'TRANSCRIPTION OF THE TABLE' in x else 'N'
)


# Function to get embeddings
def get_embedding(text_input):
    response = oai_client.embeddings.create(
        input=text_input,
        model="text-embedding-3-large"
    )
    return response.data[0].embedding


# Generate embeddings with a progress bar
embeddings = []
for text in tqdm(df['PageText'], desc='Generating Embeddings'):
    embedding = get_embedding(text)
    embeddings.append(embedding)

# Add the embeddings to the DataFrame
df['Embeddings'] = embeddings

Generating Embeddings: 100%|██████████| 49/49 [00:18<00:00,  2.61it/s]


私たちのロジックが視覚的入力を必要とするページを正しくフラグ付けしたことを確認できます。例えば、以前に調べたページ21では、Visual_Input_Neededフラグが"Y"に設定されています。

In [113]:
# Display the flag for page 21 
filtered_rows = df[df['PageNumber'] == 21]
print(filtered_rows.Visual_Input_Processed)

20    Y
Name: Visual_Input_Processed, dtype: object


#### ステップ4: Pineconeへの埋め込みのアップロード:

このセクションでは、ドキュメントの各ページに対して生成した埋め込みをPineconeにアップロードします。埋め込みと併せて、ページ番号、テキストコンテンツ、画像パス、ページにグラフィックが含まれているかどうかなど、各ページを説明する関連するメタデータタグも含めます。

**ステップの詳細:**

**1. メタデータフィールドの作成:**
メタデータにより、より細かい検索の実行、ベクトルに関連するテキストや画像の検索、ベクトルデータベース内でのフィルタリングが可能になります。
* pageId: document_idとpageNumberを組み合わせて、各ページの一意の識別子を作成します。これを埋め込みの一意の識別子として使用します。
* pageNumber: ドキュメント内の数値ページ番号。
* text: ページから抽出されたテキストコンテンツ。
* ImagePath: ページに関連する画像のファイルパス。
* GraphicIncluded: ページに視覚的処理が必要なグラフィック要素が含まれているかどうかを示すブール値またはフラグ。

**2. 埋め込みのアップロード:**
Pinecone APIを使用して、`upsert_vector`関数で値を「upsert」します -

* 一意の識別子
* 埋め込み
* 上記で定義されたメタデータ

注意: 「Upsert」は「update」と「insert」の単語を組み合わせたものです。データベース操作において、upsertは既存のレコードが存在する場合は更新し、存在しない場合は新しいレコードを挿入するアトミック操作です。これは、挿入や更新のための個別のチェックを実行することなく、データベースが最新のデータを持つことを確実にしたい場合に特に有用です。

In [114]:
# reload the index from Pinecone 
index = pc.Index(index_name)

# Create a document ID prefix 
document_id = 'WB_Report'


# Define the async function correctly
def upsert_vector(identifier, embedding, metadata):
    try:
        index.upsert([
            {
                'id': identifier,
                'values': embedding,
                'metadata': metadata
            }
        ])
    except Exception as e:
        print(f"Error upserting vector with ID {identifier}: {e}")
        raise


for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc='Uploading to Pinecone'):
    pageNumber = row['PageNumber']

    # Create meta-data tags to be added to Pinecone 
    metadata = {
        'pageId': f"{document_id}-{pageNumber}",
        'pageNumber': pageNumber,
        'text': row['PageText'],
        'ImagePath': row['ImagePath'],
        'GraphicIncluded': row['Visual_Input_Processed']
    }

    upsert_vector(metadata['pageId'], row['Embeddings'], metadata)


Uploading to Pinecone: 100%|██████████| 49/49 [00:08<00:00,  5.93it/s]


[Pinecone](https://app.pinecone.io/)のIndexesリストに移動すると、メタデータと共にデータベースにアップサートされたベクトルを確認できるはずです。

### ステップ5: 関連ページのセマンティック検索の実行：
このセクションでは、ユーザーの質問に答える最も関連性の高いページを文書内から見つけるためのセマンティック検索を実装します。このアプローチは、Pineconeベクトルデータベースに保存された埋め込みを使用して、ユーザーのクエリとコンテンツのセマンティック類似性に基づいてページを取得します。これにより、テキストコンテンツを効果的に検索し、ユーザーの質問に答えるためのコンテキストとしてGPT-4oに提供することができます。

**ステップの詳細：**  

**1. ユーザーの質問に対する埋め込みの生成** 

* OpenAIの埋め込みモデルを使用して、ユーザーの質問の高次元ベクトル表現を生成します。
* このベクトルは質問のセマンティックな意味を捉え、保存された埋め込みに対して効率的な類似性検索を実行できるようにします。
* 埋め込みは、正確な単語が一致しない場合でも、検索クエリが文書のコンテンツとセマンティックに整合することを保証するために重要です。

**2. 関連ページのPineconeインデックスへのクエリ** 

* 生成された埋め込みを使用して、Pineconeインデックスにクエリを実行し、最も関連性の高いページを見つけます。
* Pineconeは、質問の埋め込みとベクトルデータベースに保存された埋め込みを`cosine`類似性を使用して比較することで類似性検索を実行します。思い出していただければ、ステップ1でPineconeデータベースを作成した際に、これを`metric`パラメータとして設定しました。
* 取得する上位マッチの数を指定します。これは通常、カバレッジと関連性のバランスに基づいて決定されます。例えば、上位3-5ページを取得することで、モデルに過度なコンテキストを与えることなく包括的な回答を提供するのに十分な場合が多いです。

**3. マッチしたページのメタデータをコンパイルしてコンテキストを提供** 

* 関連する埋め込みが特定されると、抽出されたテキストやページ番号を含む関連メタデータを収集します。
* このメタデータは、GPT-4oに提供するコンテキストを構造化するために不可欠です。
* また、コンパイルされた情報をJSONとしてフォーマットし、LLMが解釈しやすくします。

**4. GPT-4oモデルを使用した回答の生成** 

* 最後に、コンパイルされたコンテキストをGPT-4oに渡します。
* モデルはコンテキストを使用して、ユーザーの質問に対する情報豊富で一貫性があり、文脈的に関連性の高い回答を生成します。
* 取得されたコンテキストは、文書から関連情報にアクセスできるため、LLMがより高い精度で質問に答えるのに役立ちます。

In [115]:
import json


# Function to get response to a user's question 
def get_response_to_question(user_question, pc_index):
    # Get embedding of the question to find the relevant page with the information 
    question_embedding = get_embedding(user_question)

    # get response vector embeddings 
    response = pc_index.query(
        vector=question_embedding,
        top_k=2,
        include_values=True,
        include_metadata=True
    )

    # Collect the metadata from the matches
    context_metadata = [match['metadata'] for match in response['matches']]

    # Convert the list of metadata dictionaries to prompt a JSON string
    context_json = json.dumps(context_metadata, indent=3)

    prompt = f"""You are a helpful assistant. Use the following context and images to answer the question. In the answer, include the reference to the document, and page number you found the information on between <source></source> tags. If you don't find the information, you can say "I couldn't find the information"

    question: {user_question}
    
    <SOURCES>
    {context_json}
    </SOURCES>
    """

    # Call completions end point with the prompt 
    completion = oai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt}
        ]
    )

    return completion.choices[0].message.content

では、図表からの情報を必要とする質問を提示してみましょう。この場合、関連する詳細は円グラフ内に見つけることができます。

In [116]:
question = "What percentage was allocated to social protections in Western and Central Africa?"
answer = get_response_to_question(question, index)

print(answer)

Social protection was allocated 8% of the total lending in Western and Central Africa in fiscal 2024. <source>WB_Report-13, page 13</source>


より挑戦的にするために、表に示された情報の解釈を必要とする質問をしてみましょう。ステップ2では、GPT-4oのビジョンモダリティを使用してこの情報を抽出しました。

In [117]:
question = "What was the increase in access to electricity between 2000 and 2012 in Western and Central Africa?"
answer = get_response_to_question(question, index)

print(answer)

The increase in access to electricity between 2000 and 2012 in Western and Central Africa was from 34.1% to 44.1%, which is an increase of 10 percentage points. 

<source>WB_Report-13, page 13</source>


このアプローチはうまく機能しました。しかし、複雑な工学図面などのように、画像やグラフィックに埋め込まれた情報がテキストに変換される際に忠実性を失う場合があります。

GPT-4o Visionモダリティを使用することで、ページの画像を直接モデルのコンテキストとして渡すことができます。次のセクションでは、画像入力を使用してモデルの応答精度を向上させる方法を探ります。

### ステップ6: 視覚的コンテンツを含むページの処理（オプションステップ）：
メタデータが画像、グラフィック、または表の存在を示している場合、抽出されたテキストの代わりに画像をGPT-4oのコンテキストとして渡すことができます。このアプローチは、視覚的情報のテキスト記述だけでは十分にコンテキストを伝えられない場合に有用です。これは、エンジニアリング図面や複雑な図表などの複雑なグラフィックの場合に該当することがあります。

**ステップの詳細：**

このステップとステップ5の違いは、埋め込みに対して`Visual_Input_Processed`フラグが設定されているかを識別する追加のロジックを追加したことです。その場合、テキストをコンテキストとして渡す代わりに、GPT-4oのビジョンモダリティを使用してページの画像をコンテキストとして渡します。

注意：このアプローチは、画像入力の処理がより多くのリソースを必要とし、コストも高いため、レイテンシとコストの両方を増加させます。したがって、上記のステップ5で説明したテキストのみのモダリティでは望ましい結果を得られない場合にのみ使用すべきです。

In [120]:
import base64
import json


def get_response_to_question_with_images(user_question, pc_index):
    # Get embedding of the question to find the relevant page with the information 
    question_embedding = get_embedding(user_question)

    # Get response vector embeddings 
    response = pc_index.query(
        vector=question_embedding,
        top_k=3,
        include_values=True,
        include_metadata=True
    )

    # Collect the metadata from the matches
    context_metadata = [match['metadata'] for match in response['matches']]

    # Build the message content
    message_content = []

    # Add the initial prompt
    initial_prompt = f"""You are a helpful assistant. Use the text and images provided by the user to answer the question. You must include the reference to the page number or title of the section you the answer where you found the information. If you don't find the information, you can say "I couldn't find the information"

    question: {user_question}
    """
    
    message_content.append({"role": "system", "content": initial_prompt})
    
    context_messages = []

    # Process each metadata item to include text or images based on 'Visual_Input_Processed'
    for metadata in context_metadata:
        visual_flag = metadata.get('GraphicIncluded')
        page_number = metadata.get('pageNumber')
        page_text = metadata.get('text')
        message =""

        if visual_flag =='Y':
            # Include the image
            print(f"Adding page number {page_number} as an image to context")
            image_path = metadata.get('ImagePath', None)
            try:
                base64_image = encode_image(image_path)
                image_type = 'jpeg'
                # Prepare the messages for the API call
                context_messages.append({
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/{image_type};base64,{base64_image}"
                    },
                })
            except Exception as e:
                print(f"Error encoding image at {image_path}: {e}")
        else:
            # Include the text
            print(f"Adding page number {page_number} as text to context")
            context_messages.append({
                    "type": "text",
                    "text": f"Page {page_number} - {page_text}",
                })
        
                # Prepare the messages for the API call
        messages =  {
                "role": "user",
                "content": context_messages
        }
    
    message_content.append(messages)

    completion = oai_client.chat.completions.create(
    model="gpt-4o",
    messages=message_content
    )

    return completion.choices[0].message.content

ステップ5でテキストのみのセマンティック検索について尋ねたのと同じ質問を検証してみましょう。GPT-4oモデルが質問に答えるための関連情報を含む図表を特定できることがわかります。

In [121]:
question = "What percentage was allocated to social protections in Western and Central Africa?"
answer = get_response_to_question_with_images(question, index)

print(answer)

Adding page number 13.0 as an image to context
Adding page number 12.0 as an image to context
Adding page number 11.0 as an image to context
The percentage allocated to social protection in Western and Central Africa is 8% (Figure 2: Western and Central Africa; IBRD and IDA Lending by Sector).


それでは、テキストのみのモダリティでは答えることができない可能性のある質問をしてみましょう。例えば、文書内の関連する画像を見つけて、その画像を説明するような質問です。

In [124]:
question = "Can you find the image associated with digital improvements and describe what you see in the images?"
answer = get_response_to_question_with_images(question, index)

print(answer)

Adding page number 32.0 as an image to context
Adding page number 10.0 as an image to context
Adding page number 4.0 as an image to context
### Image Descriptions

1. **Page 60-61 (Digital Section)**:
   - **Left Side**: A person is sitting and working on a laptop, holding a smartphone. The setting seems informal, possibly in a small office or a cafe.
   - **Text**: Discussion on scaling digital development, thought leadership, partnerships, and establishment of a Digital Vice Presidency unit for digital transformation efforts.

2. **Page 16-17 (Eastern and Southern Africa Section)**:
   - **Right Side**: A group of people standing on a paved street, some using mobile phones. It seems to be a casual, evening setting.
   - **Text**: Information about improving access to electricity in Rwanda and efforts for education and other services in Eastern and Southern Africa.

3. **Page 4-5 (Driving Action, Measuring Results)**:
   - **Images**: Various circular images and icons accompany text h

### まとめ

このクックブックでは、画像、グラフィック、表が豊富に含まれた文書に対するRetrieval-Augmented Generation（RAG）システムの強化に取り組みました。従来のRAGモデルは、テキストデータには習熟していますが、視覚的要素を通じて伝達される豊富な情報を見落とすことがよくあります。ビジョンモデルを統合し、メタデータタギングを活用することで、このギャップを埋め、AIが視覚的コンテンツを効果的に解釈し活用できるようにしました。

まず、Pineconeを使用してベクトルストアを設定し、ベクトル埋め込みの効率的な保存と検索の基盤を確立しました。PDFを解析し、GPT-4oのビジョンモダリティを使用して視覚情報を抽出することで、文書ページを関連するテキストに変換できました。埋め込みを生成し、視覚的コンテンツを含むページにフラグを立てることで、ベクトルストア内に堅牢なメタデータフィルタリングシステムを作成しました。

これらの埋め込みをPineconeにアップロードすることで、RAG処理ワークフローとのシームレスな統合が促進されました。セマンティック検索を通じて、ユーザークエリに一致する関連ページを取得し、テキストと視覚情報の両方が考慮されることを確保しました。視覚的コンテンツを含むページをビジョンモデルに渡すことで処理することにより、特に画像や表に依存するクエリに対する回答の精度と深度が向上しました。

世界銀行の**A Better Bank for a Better World: Annual Report 2024**を指針となる例として使用し、これらの技術がどのように組み合わさって複雑な文書を処理し解釈するかを実証しました。このアプローチは、ユーザーに提供される情報を豊かにするだけでなく、より包括的で正確な回答を提供することで、ユーザーの満足度とエンゲージメントを大幅に向上させます。

このクックブックで概説された概念に従うことで、複雑な視覚的要素を含む文書を処理し解釈できるRAGシステムを構築する準備が整いました。この進歩により、視覚データが重要な役割を果たすさまざまな領域でのAIアプリケーションの新たな可能性が開かれます。